# IRIS Model Training

**Color Pattern Recognition Specialist**

This notebook trains the IRIS model, which specializes in:
- Color relationships and mappings
- Conditional color transformations
- Pattern-based color rules
- Color embedding techniques
- Context-dependent recoloring

## Setup Environment

In [ ]:
# Clone repository and install dependencies
!git clone https://github.com/AutomataControls/AutomataNexus_Olympus_AGI2.git /content/AutomataNexus_Olympus_AGI2
!cd /content/AutomataNexus_Olympus_AGI2 && pip install -r requirements.txt -q

In [ ]:
# Verify environment
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Initialize IRIS Iterative Training Controller
import json
import os
import datetime
from typing import Dict, List

class IrisIterativeTrainer:
    def __init__(self, base_dir='/content/AutomataNexus_Olympus_AGI2'):
        self.base_dir = base_dir
        self.iteration_log_file = f"{base_dir}/arc_models_v4/iris_iterations.json"
        self.current_params = self.load_default_params()
        self.iterations = self.load_iteration_history()
    
    def load_default_params(self) -> Dict:
        return {
            'learning_rate': 0.005,
            'batch_size': 512,
            'epochs_per_stage': 100,
            'color_embedding_dim': 64,
            'color_attention_heads': 4,
            'mapping_weight': 1.5,
            'transformation_penalty': 0.5,
            'exact_match_bonus': 5.0,
            'gradient_accumulation_steps': 4,
            'color_augmentation': True,
            'embedding_dropout': 0.15,
            'target_accuracy': 95.0,
            'use_mept': True,
            'use_leap': True,
            'use_prism': True
        }
    
    def load_iteration_history(self) -> List[Dict]:
        if os.path.exists(self.iteration_log_file):
            with open(self.iteration_log_file, 'r') as f:
                return json.load(f)
        return []
    
    def save_iteration_history(self):
        os.makedirs(os.path.dirname(self.iteration_log_file), exist_ok=True)
        with open(self.iteration_log_file, 'w') as f:
            json.dump(self.iterations, f, indent=2)
    
    def log_iteration(self, params: Dict, results: Dict):
        iteration = {
            'iteration': len(self.iterations) + 1,
            'timestamp': datetime.datetime.now().isoformat(),
            'parameters': params.copy(),
            'results': results.copy()
        }
        self.iterations.append(iteration)
        self.save_iteration_history()
    
    def get_best_iteration(self) -> Dict:
        if not self.iterations:
            return None
        return max(self.iterations, key=lambda x: x['results'].get('best_exact', 0))
    
    def suggest_next_params(self) -> Dict:
        if len(self.iterations) < 2:
            return self.current_params.copy()
        
        best = self.get_best_iteration()
        latest = self.iterations[-1]
        
        suggestions = best['parameters'].copy() if best else self.current_params.copy()
        
        if latest and best:
            latest_exact = latest['results'].get('best_exact', 0)
            best_exact = best['results'].get('best_exact', 0)
            
            if latest_exact < best_exact * 0.85:
                suggestions['learning_rate'] *= 0.7
                suggestions['mapping_weight'] = min(2.5, suggestions['mapping_weight'] * 1.2)
                suggestions['embedding_dropout'] = max(0.1, suggestions['embedding_dropout'] * 0.9)
                suggestions['color_attention_heads'] = min(8, suggestions['color_attention_heads'] + 1)
            elif latest_exact > best_exact * 1.05:
                suggestions['learning_rate'] = min(0.006, suggestions['learning_rate'] * 1.05)
                suggestions['color_embedding_dim'] = min(96, suggestions['color_embedding_dim'] + 16)
        
        return suggestions
    
    def display_history(self):
        if not self.iterations:
            print("No IRIS iterations found. Starting fresh!")
            return
        
        print("IRIS Training History:")
        print("-" * 85)
        for i, iteration in enumerate(self.iterations):
            exact = iteration['results'].get('best_exact', 0)
            loss = iteration['results'].get('best_val_loss', float('inf'))
            lr = iteration['parameters'].get('learning_rate', 0)
            mapping_weight = iteration['parameters'].get('mapping_weight', 0)
            embed_dim = iteration['parameters'].get('color_embedding_dim', 0)
            timestamp = iteration['timestamp'][:16]
            
            status = "BEST" if iteration == self.get_best_iteration() else ""
            print(f"{status} Iter {i+1}: {exact:.2f}% exact | Loss: {loss:.4f} | LR: {lr:.4f} | Map: {mapping_weight:.1f} | Emb: {embed_dim} | {timestamp}")
        
        print("-" * 85)
        best = self.get_best_iteration()
        if best:
            print(f"Best: Iteration {best['iteration']} with {best['results']['best_exact']:.2f}% exact match")

iris_trainer = IrisIterativeTrainer()
suggested_params = iris_trainer.suggest_next_params()
iris_trainer.display_history()
print("\nIRIS trainer initialized. You can now run the training cells below.")

## Start IRIS Training

Training configuration:
- **Architecture**: Color Attention + Mapping Networks
- **Parameters**: ~0.9M
- **Specialization**: Color patterns
- **Focus**: Color embedding and mapping rule extraction

In [ ]:
# Train IRIS model
!cd /content/AutomataNexus_Olympus_AGI2 && python scripts/training/train_iris.py

## Monitor Training Progress

In [ ]:
# IRIS Iterative Training
import subprocess
import tempfile
import os

if 'iris_trainer' not in globals():
    print("Error: Run the trainer initialization cell first!")
    raise NameError("iris_trainer not defined")

LEARNING_RATE = suggested_params.get('learning_rate', 0.005)
BATCH_SIZE = suggested_params.get('batch_size', 512)
EPOCHS_PER_STAGE = suggested_params.get('epochs_per_stage', 100)
TRANSFORMATION_PENALTY = 0.5
EXACT_MATCH_BONUS = 5.0
GRADIENT_ACCUMULATION_STEPS = 4
USE_MEPT = True
USE_LEAP = True
USE_PRISM = True
TARGET_ACCURACY = 95.0

print(f"Starting IRIS training iteration {len(iris_trainer.iterations) + 1}")
print(f"Parameters: LR={LEARNING_RATE}, BS={BATCH_SIZE}, Epochs/Stage={EPOCHS_PER_STAGE}")
print(f"Loss weights: Transform penalty={TRANSFORMATION_PENALTY}, Exact bonus={EXACT_MATCH_BONUS}")
print(f"Features: MEPT={USE_MEPT}, LEAP={USE_LEAP}, PRISM={USE_PRISM}")
print(f"Target accuracy: {TARGET_ACCURACY}%")

modified_script = f"""
import sys
sys.path.append('/content/AutomataNexus_Olympus_AGI2')
sys.path.append('/content/AutomataNexus_Olympus_AGI2/src')

import scripts.training.train_iris as train_module
train_module.LEARNING_RATE = {LEARNING_RATE}
train_module.BATCH_SIZE = {BATCH_SIZE}
train_module.EPOCHS_PER_STAGE = {EPOCHS_PER_STAGE}
train_module.TRANSFORMATION_PENALTY = {TRANSFORMATION_PENALTY}
train_module.EXACT_MATCH_BONUS = {EXACT_MATCH_BONUS}
train_module.GRADIENT_ACCUMULATION_STEPS = {GRADIENT_ACCUMULATION_STEPS}
train_module.USE_MEPT = {USE_MEPT}
train_module.USE_LEAP = {USE_LEAP}
train_module.USE_PRISM = {USE_PRISM}

try:
    import scripts.training.stage0_exact_match_boost as boost_module
    original_inject = boost_module.inject_exact_match_training
    def inject_with_target(*args, **kwargs):
        kwargs['target_accuracy'] = {TARGET_ACCURACY}
        return original_inject(*args, **kwargs)
    boost_module.inject_exact_match_training = inject_with_target
except ImportError:
    print("Warning: stage0_exact_match_boost not available")

train_module.train_iris()
"""

with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(modified_script)
    script_path = f.name

try:
    print("Training timeout: 8 hours")
    result = subprocess.run(
        ['python', script_path],
        cwd='/content/AutomataNexus_Olympus_AGI2',
        capture_output=True,
        text=True,
        timeout=28800
    )
    
    print("IRIS training output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
finally:
    if os.path.exists(script_path):
        os.unlink(script_path)

In [ ]:
# 🔧 IRIS Iterative Training Controller
import json
import os
import datetime
from typing import Dict, List

class IrisIterativeTrainer:
    def __init__(self, base_dir='/content/AutomataNexus_Olympus_AGI2'):
        self.base_dir = base_dir
        self.iteration_log_file = f"{base_dir}/arc_models_v4/iris_iterations.json"
        self.current_params = self.load_default_params()
        self.iterations = self.load_iteration_history()
    
    def load_default_params(self) -> Dict:
        return {
            'learning_rate': 0.004,  # Moderate for color learning
            'batch_size': 384,       # Medium batch for color patterns
            'epochs_per_stage': 110, # Balanced epochs for color convergence
            'color_embedding_dim': 64,  # IRIS-specific
            'color_attention_heads': 4, # IRIS-specific
            'mapping_weight': 1.5,   # Color mapping loss weight
            'transformation_penalty': 0.6,
            'exact_match_bonus': 5.5,
            'gradient_accumulation_steps': 6,
            'color_augmentation': True,   # IRIS-specific
            'embedding_dropout': 0.15     # IRIS-specific
        }
    
    def load_iteration_history(self) -> List[Dict]:
        if os.path.exists(self.iteration_log_file):
            with open(self.iteration_log_file, 'r') as f:
                return json.load(f)
        return []
    
    def save_iteration_history(self):
        os.makedirs(os.path.dirname(self.iteration_log_file), exist_ok=True)
        with open(self.iteration_log_file, 'w') as f:
            json.dump(self.iterations, f, indent=2)
    
    def log_iteration(self, params: Dict, results: Dict):
        iteration = {
            'iteration': len(self.iterations) + 1,
            'timestamp': datetime.datetime.now().isoformat(),
            'parameters': params.copy(),
            'results': results.copy()
        }
        self.iterations.append(iteration)
        self.save_iteration_history()
    
    def get_best_iteration(self) -> Dict:
        if not self.iterations:
            return None
        return max(self.iterations, key=lambda x: x['results'].get('best_exact', 0))
    
    def suggest_next_params(self) -> Dict:
        if len(self.iterations) < 2:
            return self.current_params
        
        best = self.get_best_iteration()
        latest = self.iterations[-1]
        
        suggestions = best['parameters'].copy()
        
        # IRIS-specific adaptive suggestions for color learning
        latest_exact = latest['results'].get('best_exact', 0)
        best_exact = best['results'].get('best_exact', 0)
        
        if latest_exact < best_exact * 0.85:  # Performance dropped
            # For color tasks, adjust embedding and attention
            suggestions['learning_rate'] *= 0.7
            suggestions['mapping_weight'] = min(2.5, suggestions['mapping_weight'] * 1.2)
            suggestions['embedding_dropout'] = max(0.1, suggestions['embedding_dropout'] * 0.9)
            suggestions['color_attention_heads'] = min(8, suggestions['color_attention_heads'] + 1)
        elif latest_exact > best_exact * 1.05:  # Good improvement
            # Increase color complexity
            suggestions['learning_rate'] = min(0.006, suggestions['learning_rate'] * 1.05)
            suggestions['color_embedding_dim'] = min(96, suggestions['color_embedding_dim'] + 16)
        
        return suggestions
    
    def display_history(self):
        if not self.iterations:
            print("No IRIS iterations found.")
            return
        
        print("📈 IRIS Training History:")
        print("-" * 85)
        for i, iteration in enumerate(self.iterations):
            exact = iteration['results'].get('best_exact', 0)
            loss = iteration['results'].get('best_val_loss', float('inf'))
            lr = iteration['parameters'].get('learning_rate', 0)
            mapping_weight = iteration['parameters'].get('mapping_weight', 0)
            embed_dim = iteration['parameters'].get('color_embedding_dim', 0)
            timestamp = iteration['timestamp'][:16]
            
            status = "🟢 BEST" if iteration == self.get_best_iteration() else "⚪"
            print(f"{status} Iter {i+1}: {exact:.2f}% exact | Loss: {loss:.4f} | LR: {lr:.4f} | Map: {mapping_weight:.1f} | Emb: {embed_dim} | {timestamp}")
        
        print("-" * 85)
        best = self.get_best_iteration()
        if best:
            print(f"🏆 Best: Iteration {best['iteration']} with {best['results']['best_exact']:.2f}% exact match")

# Initialize IRIS trainer
iris_trainer = IrisIterativeTrainer()
iris_trainer.display_history()

In [ ]:
# Clear GPU memory if needed
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
print(f"Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1e9:.2f} GB")

# Load and evaluate IRIS model
import torch
import json
import glob

model_files = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/iris_best.pt')

if model_files:
    model_file = model_files[0]
    checkpoint = torch.load(model_file, map_location='cpu')
    
    print("IRIS Training Results:")
    print(f"  Best Validation Exact Match: {checkpoint.get('val_exact', 'N/A'):.2f}%")
    print(f"  Training Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"  Stage: {checkpoint.get('stage', 'N/A')}")
    print(f"  Model Size: {os.path.getsize(model_file) / (1024**2):.1f} MB")
else:
    print("IRIS model not found. Training may still be in progress.")

report_files = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/iris_training_report_*.json')
if report_files:
    latest_report = sorted(report_files)[-1]
    with open(latest_report, 'r') as f:
        report = json.load(f)
    print(f"\nLatest Training Report:")
    print(f"  Best Exact Match: {report.get('best_exact', 'N/A'):.2f}%")
    print(f"  Final Validation Loss: {report.get('best_val_loss', 'N/A'):.4f}")
    print(f"  Training Duration: {report.get('training_time', 'N/A')}")

## Troubleshooting

If you encounter issues:

In [ ]:
# Clear GPU memory if needed
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
print(f"Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1e9:.2f} GB")

---

## Training Complete!

Your IRIS model has been trained and saved to `/content/AutomataNexus_Olympus_AGI2/arc_models_v4/`

**IRIS Specialization:**
- Color pattern recognition
- Advanced color embedding techniques
- Conditional color transformations

**AutomataNexus OLYMPUS AGI2** - *Where Neural Networks Meet Symbolic Logic*